# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = '../Output_data/CityWeatherData.csv'
city_weather = pd.read_csv(city_weather)
city_weather = city_weather.dropna()
city_weather

,City,Country,Lat,Lng,Max_Temp,Humidity,Cloudiness,Windspeed,Date
0,dikson,RU,73.51,80.55,41.45,83.0,0.0,15.59,1.590129e+09
1,sao joao da barra,BR,-21.64,-41.05,69.55,85.0,0.0,9.62,1.590129e+09
2,tuktoyaktuk,CA,69.45,-133.04,32.31,98.0,100.0,9.08,1.590129e+09
3,tiksi,RU,71.69,128.87,32.56,95.0,95.0,4.09,1.590129e+09
4,punta arenas,CL,-53.15,-70.92,41.00,80.0,75.0,8.05,1.590129e+09
...,...,...,...,...,...,...,...,...,...
617,srinagar,IN,34.08,74.82,78.19,31.0,100.0,1.99,1.590130e+09
618,katsuura,JP,35.13,140.30,66.99,100.0,75.0,12.75,1.590130e+09
619,filadelfia,PY,-22.35,-60.03,63.99,93.0,98.0,10.25,1.590130e+09
620,tecoanapa,MX,16.52,-98.75,77.27,81.0,0.0,4.23,1.590130e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

humidity = city_weather["Humidity"].astype(float)
locations = city_weather[["Lat", "Lng"]]

In [4]:
locations = city_weather[["Lat", "Lng"]]
humidity = city_weather['Humidity'].astype(int)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10,point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = city_weather.loc[(city_weather["Max_Temp"] > 70) & 
                                 (city_weather["Max_Temp"] < 80) & 
                                 (city_weather["Cloudiness"] == 0)]
ideal_weather = ideal_weather[['City','Country','Lat','Lng']]
ideal_weather = ideal_weather.reset_index()
ideal_weather = ideal_weather.drop(columns=['index'])
ideal_weather.head()

,City,Country,Lat,Lng
0,busselton,AU,-33.65,115.33
1,teguldet,RU,57.31,88.17
2,carnarvon,AU,-24.87,113.63
3,dunda,TZ,-8.12,34.25
4,cabo san lucas,MX,22.89,-109.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
ideal_weather['Hotel']= ''
hotel_df = ideal_weather

target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params)

In [7]:
hotel_name = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()

    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [8]:
hotel_df['Hotel Name']= hotel_name
hotel_df.head()

,City,Country,Lat,Lng,Hotel,Hotel Name
0,busselton,AU,-33.65,115.33,,Broadwater Resort
1,teguldet,RU,57.31,88.17,,Gostinitsa Kedr
2,carnarvon,AU,-24.87,113.63,,Coral Coast Tourist Park
3,dunda,TZ,-8.12,34.25,,NaN
4,cabo san lucas,MX,22.89,-109.91,,Holiday Inn Express Cabo San Lucas


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))